## Тестовое задание Oliinyk Bohdan
Мной было разработано две варианта решения этой задачи.
 1. С помощью готовой модели **rubert-base-cased** 
 https://huggingface.co/DeepPavlov/rubert-base-cased
 *Насколько я понял, то выделенные программой ключевые пункты диалога-должны свидетельстовать об основных проблемах пациента. Поэтому результаты на основе заранее натренированой модели показались мне нерепрезентативными и решено было сделать собственную модель, которая присваивает сначала каждому слову веса(перед этим удалаются все "стопслова", пунктуация, цифры), и на основе них "взвешиваются" предложения.*
 2. Собственная модель

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install bert-extractive-summarizer

### Rubert-base-cased Model

In [ ]:
!pip install nltk
import nltk 
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from summarizer import Summarizer

In [ ]:
import urllib.request

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

# Load model
custom_config = AutoConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#opening file
from urllib.request import urlopen
text = urlopen("https://drive.google.com/uc?id=18HkjO-2kSrqOrCsPR59OQZ4Rn41C3IwW&export=download")
text = str(text.read(), 'utf-8')
#generating results
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
n = int(input("Введите количество предложений: "))
result = model(text, num_sentences=n)


Введите количество предложений: 10


In [ ]:
#printing results
sentences = sent_tokenize(result) 
sen_num = 0
for sen in sentences:
    sen_num+=1
    print(sen_num, sen,"\n")
    



1 ﻿Психолог: Добрый день, меня зовут Ирина. 

2 Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас? 

3 Я реально уже думала что можно и начать инвестировать но блин, реально никогда не знаешь, теперь думаю что можно было и пол года назад параллельно с подушкой все делать, но так иногда радует что хоть стабильность в деньгах есть, я все таки смогла научиться управлять немного своими финансами. 

4 Сейчас у вас нет основной работы для вас большой стресс искать новую, еще вы хотите вернуться в Киев, но не можете решить для себя стоит ли это сейчас делать, также вам хочется получить поддержку от мужа, но вы не можете получить той помощи на которую рассчитываете. 

5 Ощущение, что теряешь время, которого и так постоянно мало, и параллельно же ничего не улучшается, тебе еще наваливается сверху, и нужно разгребать все одновременно. 

6 Психолог: Расскажите подробнее, я так понимаю у вас разные точки зрения на какую то ситуацию с мужем? 

7 А ракета может п

### Собственная модель

In [ ]:
import nltk 
from nltk.corpus import stopwords  

nltk.download('stopwords')

from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#opening file
text = urlopen("https://drive.google.com/uc?id=18HkjO-2kSrqOrCsPR59OQZ4Rn41C3IwW&export=download")
text = str(text.read(), 'utf-8')


In [ ]:
#Creating StopWord list(with punctuation and numbers)
stopWords = list(stopwords.words("russian"))+list(punctuation)+list([0,1,2,3,4,5,6,7,8,9])

In [ ]:
#Tokenizating text, and creating a word frequency dictionary
words = word_tokenize(text) 

freqTable={}
   
for word in words: 
    word = word.lower() 
    if word not in stopWords:
        if word in freqTable: 
            freqTable[word] += 1
        else: 
            freqTable[word] = 1
print(freqTable.items())

dict_items([('\ufeffпсихолог', 1), ('добрый', 2), ('день', 2), ('зовут', 1), ('ирина', 1), ('ольга', 9), ('вами', 2), ('сегодня', 1), ('назначена', 1), ('консультация', 2), ('надеюсь', 1), ('наш', 1), ('разговор', 1), ('приятным', 1), ('полезным', 1), ('расскажите', 5), ('каким', 1), ('запросом', 1), ('пришли', 1), ('нашу', 1), ('первую', 1), ('встречу', 2), ('это', 11), ('первая', 1), ('жизни', 2), ('особо', 2), ('понимаю', 8), ('именно', 4), ('нужно', 10), ('говорить', 1), ('накопилось', 1), ('достаточно', 1), ('просто', 17), ('хотелось', 2), ('поговорить', 1), ('происходит', 4), ('моей', 1), ('жизнью', 1), ('таки', 3), ('война', 1), ('переезды', 1), ('куча', 1), ('короче', 6), ('кто-то', 1), ('услышал', 1), ('помог', 2), ('немного', 3), ('разобраться', 2), ('психолог', 9), ('действительно', 1), ('многие', 1), ('проживают', 1), ('события', 2), ('которым', 1), ('сложно', 3), ('готовым', 1), ('давайте', 3), ('начнем', 2), ('понемногу', 2), ('разбираться', 2), ('находитесь', 1), ('каких

In [ ]:
#Tokenizating text, and creating a dictionary with weight of each sentence
sentences = sent_tokenize(text) 
sentence_weight = {} 
for sentence in sentences: 
    for word, freq in freqTable.items():
        if word in sentence.lower(): 
            if sentence in sentence_weight:     
                sentence_weight[sentence] += freq 
            else: 
                sentence_weight[sentence] = freq 
sentence_weight

{'А можно просто было все спокойно сделать вовремя, потому что есть понимание того что вовремя все таки существует, ну или более удачное время, типа чего не пользоваться ситуацией?': 50,
 'А ракета может прилететь куда угодно или не прилететь.': 4,
 'А теперь читаю про блекауты, теории заговора, считаю остаток бюджета и на сколько мне хватит денег при разных условиях жизни, думаю нужно ли пытаюсь выделить деньги на английский, мне постоянно прилетают дополнительные расходы.': 57,
 'А я не хочу ехать одна, я хочу чтобы мы вдвоем поехали, потому что ну что  я там буду делать одна, мне реально надо чтобы мы или вместе были здесь или там.': 51,
 'Блин я уже иногда думаю, а если бы мы не были вместе если мне не надо было бы постоянно ждать, где бы я уже была?': 29,
 'Ведь мне нравилось жить в том потоке, что происходил.': 12,
 'Вот что происходит с вами, когда вам не нужно никого ждать?': 30,
 'Вот я жду например, что мы купим билеты и полетим куда-то на отдых, мы все равно покупаем и летим

In [ ]:
sumValues = 0
for sentence in sentence_weight: 
    sumValues += sentence_weight[sentence] 

average = int(sumValues / len(sentence_weight)) 

summary = '' 
counter=0
n = int(input("Введите количество предложений: "))
for sentence in sentences: 
    if counter!=n:
        if (sentence in sentence_weight) and (sentence_weight[sentence] > (1.1* average)): 
            summary += " " + sentence 
            counter+=1
sentences_my = sent_tokenize(summary) 
sen_num_my = 0
for sen in sentences_my:
    sen_num_my+=1
    print(sen_num_my,sen,"\n")

Введите количество предложений: 9
1  Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. 

2 Деньги немного есть, ээээ ну там на пару месяцев хватит, но нужно искать работу. 

3 Потерять финансовую стабильность мне реально тяжело, ждать что что-то наладится, но ничего не наладится же. 

4 Я реально уже думала что можно и начать инвестировать но блин, реально никогда не знаешь, теперь думаю что можно было и пол года назад параллельно с подушкой все делать, но так иногда радует что хоть стабильность в деньгах есть, я все таки смогла научиться управлять немного своими финансами. 

5 Наверно скорее всего нужно будет учить английский, вряд ли украинские компании сейчас смогут дать хорошую оплату, надо искать что-то онлайн, стабильное. 

6 Короче искать работу это мне вообще как настоящее испытание, ну и не думаю что без английского я кому-то